# **Generate text simplifications using fine-tuned mBART model**

# Mount gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Create required folder structure

In [ ]:
mkdir model model/preprocess model/spm

# Download mBART 50

In [ ]:
!wget https://dl.fbaipublicfiles.com/fairseq/models/mbart50/mbart50.pretrained.tar.gz
!tar -xzvf mbart50.pretrained.tar.gz

--2022-01-08 13:08:38--  https://dl.fbaipublicfiles.com/fairseq/models/mbart50/mbart50.pretrained.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7199950118 (6.7G) [application/gzip]
Saving to: ‘mbart50.pretrained.tar.gz’

mbart50.pretrained. 100%[===================>]   6.71G  34.7MB/s    in 3m 15s  

2022-01-08 13:11:54 (35.1 MB/s) - ‘mbart50.pretrained.tar.gz’ saved [7199950118/7199950118]

mbart50.pretrained/
mbart50.pretrained/sentence.bpe.model
mbart50.pretrained/dict.af_ZA.txt
mbart50.pretrained/dict.ar_AR.txt
mbart50.pretrained/dict.az_AZ.txt
mbart50.pretrained/dict.bn_IN.txt
mbart50.pretrained/dict.cs_CZ.txt
mbart50.pretrained/dict.de_DE.txt
mbart50.pretrained/dict.en_XX.txt
mbart50.pretrained/dict.es_XX.txt
mbart50.pretrained/dict.et_EE.txt
mbart50.pretrained/dict

In [ ]:
!cp mbart50.pretrained/model.pt model/model.pt

In [ ]:
!cp mbart50.pretrained/dict.si_LK.txt model/dict.si_LK.txt

In [ ]:
!cp mbart50.pretrained/sentence.bpe.model model/sentence.bpe.model

# Install libraries

In [ ]:
!git clone https://github.com/brainsharks-fyp17/fairseq.git && \
cd fairseq && \
pip install --editable ./ -q
!pip install fastBPE sacremoses subword_nmt sentencepiece -q
!pip install sacrebleu==1.5.1

Cloning into 'fairseq'...
remote: Enumerating objects: 29383, done.
remote: Total 29383 (delta 0), reused 0 (delta 0), pack-reused 29383
Receiving objects: 100% (29383/29383), 12.50 MiB | 27.65 MiB/s, done.
Resolving deltas: 100% (21999/21999), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 90 kB 5.6 MB/s 
     |████████████████████████████████| 145 kB 28.9 MB/s 
     |████████████████████████████████| 74 kB 4.4 MB/s 
     |████████████████████████████████| 112 kB 60.2 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 895 kB 11.9 MB/s 
     |████████████████████████████████| 1.2 MB 91.8 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
  Attempting uninstall: portalocker
    Found existing installation: portalocker 2.3.2
    Uninstalling portalocker-2.3.2:

In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

# Download test datasets

In [ ]:
!gdown --id 1JsV5PJAp9BOvjydjC9D5v2o6zB11XFne

Downloading...
From: https://drive.google.com/uc?id=1JsV5PJAp9BOvjydjC9D5v2o6zB11XFne
To: /content/complex-1000.txt
100% 600k/600k [00:00<00:00, 79.4MB/s]


In [ ]:
!gdown --id 1TYUoCngG33T2O0LWGD5iMwSkzNOegPjE

Downloading...
From: https://drive.google.com/uc?id=1TYUoCngG33T2O0LWGD5iMwSkzNOegPjE
To: /content/simp1000-piyumika2.txt
100% 606k/606k [00:00<00:00, 86.7MB/s]


# SPM encode

In [ ]:
!python fairseq/scripts/spm_encode.py \
    --model model/sentence.bpe.model \
    --inputs complex-1000.txt \
    --outputs model/spm/test.bpe.si_LK

skipped 0 empty lines
filtered 0 lines


# Preprocess data

In [ ]:
DICT_s="model/dict.si_LK.txt"

!fairseq-preprocess \
  --source-lang "si_LK" \
  --target-lang "si_LK" \
  --testpref model/spm/test.bpe \
  --destdir model/preprocess \
  --srcdict $DICT_s \
  --joined-dictionary \
  --thresholdtgt 0 \
  --thresholdsrc 0 \
  --workers 70

2022-01-08 13:18:38 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='model/preprocess', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='si_LK', srcdict='model/dict.si_LK.txt', target_lang='si_LK', task='translation', tensorboard_logdir=None, testpref='model/spm/test.bpe', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, to

# Predict simplifications

In [ ]:
model="model/model.pt"
source_lang="si_LK"
target_lang="si_LK"
path_2_data="model/preprocess"

!fairseq-generate "$path_2_data" \
  --path "$model" \
  --task translation_from_pretrained_bart \
  --gen-subset test \
  --source-lang "$source_lang" \
  --target-lang "$target_lang" \
  --scoring 'sacrebleu' --remove-bpe 'sentencepiece'\
  --batch-size 32 \
  --langs "ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN,af_ZA,az_AZ,bn_IN,fa_IR,he_IL,hr_HR,id_ID,ka_GE,km_KH,mk_MK,ml_IN,mn_MN,mr_IN,pl_PL,ps_AF,pt_XX,sv_SE,sw_KE,ta_IN,te_IN,th_TH,tl_XX,uk_UA,ur_PK,xh_ZA,gl_ES,sl_SI" \
  --beam 5 \
  --nbest 1 \
  --lenpen 1.0 \
  --diverse-beam-groups -1 \
  --diverse-beam-strength 0.5 > result.txt

tcmalloc: large alloc 2443517952 bytes == 0x5573f258e000 @  0x7fe61a4fcb6b 0x7fe61a51c379 0x7fe4faf70cde 0x7fe4faf72452 0x7fe54d9e4749 0x7fe5f680dd59 0x5572de97e045 0x5572de93ec52 0x5572de9b1c25 0x5572de9acced 0x5572de93f7f3 0x5572de93f2f9 0x5572dea8635d 0x5572de9f5a0b 0x5572de93e3a1 0x5572dea2fe1d 0x5572de9b1e99 0x5572de9acced 0x5572de87ee2b 0x5572de9aefe4 0x5572de9ac9ee 0x5572de93fbda 0x5572de9ae737 0x5572de9ac9ee 0x5572de93fbda 0x5572de9ad915 0x5572de9ac9ee 0x5572de93fbda 0x5572de9ad915 0x5572de9ac9ee 0x5572de93fbda
tcmalloc: large alloc 2443517952 bytes == 0x557483fe0000 @  0x7fe61a4fcb6b 0x7fe61a51c379 0x7fe4faf70cde 0x7fe4faf72452 0x7fe54d9e4749 0x7fe5f680dd59 0x5572de97e045 0x5572de93ec52 0x5572de9b1c25 0x5572de9acced 0x5572de93f7f3 0x5572de93f2f9 0x5572dea8635d 0x5572de9f5a0b 0x5572de93e3a1 0x5572dea2fe1d 0x5572de9b1e99 0x5572de9acced 0x5572de87ee2b 0x5572de9aefe4 0x5572de9ac9ee 0x5572de93fbda 0x5572de9ae737 0x5572de9ac9ee 0x5572de93fbda 0x5572de9ad915 0x5572de9ac9ee 0x5572de93

In [ ]:
cat result.txt | grep -P "^H" |sort -V |cut -f 3- > mbart50-simp.txt